In [1]:
import json 
theme_to_token = json.load(open("../extra/theme_tokens.json", "r"))
token_to_theme = {t:m for m,t in theme_to_token.items()}

meter_to_token = json.load(open("../extra/meter_tokens.json", "r"))
token_to_meter = {t:m for m,t in meter_to_token.items()}

In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM

gpt_tokenizer = AutoTokenizer.from_pretrained('arbml/ashaar_tokenizer')
encoded_input = gpt_tokenizer(['<|psep|>'], return_tensors='pt')
model = AutoModelForCausalLM.from_pretrained('../checkpoint-450000')

2023-06-19 14:29:57.994111: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-19 14:29:58.813560: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [17]:
theme = "قصيدة غزل"
meter = "البسيط"
qafiyah = "ه"
prompt = f"{meter_to_token[meter]} {qafiyah} {theme_to_token[theme]}"
print(prompt)
encoded_input = gpt_tokenizer(prompt, return_tensors='pt')
output = model.generate(**encoded_input, max_length = 512, top_p = 3, do_sample=True)
print(encoded_input)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


<|meter_4|> ه <|theme_8|>
{'input_ids': tensor([[ 11, 109,  53,  32]]), 'attention_mask': tensor([[1, 1, 1, 1]])}


In [19]:
result = ""
prev_token = ""

for i, beam in enumerate(output[:, len(encoded_input.input_ids[0]):]):
    for token in beam:
        decoded = gpt_tokenizer.decode(token)
        if 'meter' in decoded or 'theme' in decoded:
            break
        if decoded == "<|vsep|>":
            result += "\n\n"
        elif decoded in ["<|bsep|>", "</|bsep|>"]:
            result += "\n"
        elif decoded in ['<|psep|>', '</|psep|>']:
            pass
        else:
            result += decoded
        prev_token = decoded
    else:
        break
print(theme+" "+ f"من بحر {meter} مع قافية بحر ({qafiyah})" + "\n" +result)

قصيدة غزل من بحر البسيط مع قافية بحر (ه)

ما لي أراك كأنّي مستهاماً 

بها افتخرتَ بلذاتي وأبلاها 

فلذةُ القلبِ في أحشاء مسكنها 

ولذة العيش في أنات أحلاها 

مرّت لذاك وما ماتت بغانيةٍ 

لا لطالب نفسٍ قُدّ مُرّاها 

من دبر اليومَ صفو العيش نية 

كأنه عند شكراني وسكناها 

ذا ونت نكباتُ الدهر وقعته 

سدىً أصاب غنيا عن نداماها 

في ساعة لصياحي الروح مُتئداً 

كأنني طالب عصيان مُلقاها 

ملكتُ قلبي بسلوان الفتور فها 

واويلَ قلبٍ بذاك اليوم أشقاها 

أتلك شمسي في كلّ النواظر أم 

من ذي عيون لها انفرّت مقيها 

وأم ريم خذورٌ من محاجرها 

يرمي بخد المواضي ورد 


In [14]:
from huggingface_hub import HfApi
api = HfApi()
future = api.upload_folder( # Upload in the background (non-blocking action)
    repo_id="arbml/Ashaar_modelv2",
    folder_path="checkpoint-450000",
    run_as_future=True,
)
future
future.done()
future.result() # Wait for the upload to complete (blocking action)

optimizer.pt:   0%|          | 0.00/574M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/287M [00:00<?, ?B/s]

scheduler.pt:   0%|          | 0.00/627 [00:00<?, ?B/s]

rng_state.pth:   0%|          | 0.00/14.6k [00:00<?, ?B/s]

trainer_state.json:   0%|          | 0.00/55.9M [00:00<?, ?B/s]

Upload 6 LFS files:   0%|          | 0/6 [00:00<?, ?it/s]

training_args.bin:   0%|          | 0.00/3.96k [00:00<?, ?B/s]

'https://huggingface.co/arbml/Ashaar_modelv2/tree/main/'

Running on local URL:  http://0.0.0.0:8999


KeyboardInterrupt: 

In [3]:
gr.close_all()

Closing server running on port: 8999
